In [159]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
There's a new jupysql version available (0.10.2), you're running 0.7.2. To upgrade: pip install jupysql --upgrade


In [207]:
%%sql
drop table if exists Supparts;
CREATE TABLE SupParts
(sno INTEGER NOT NULL,
 pno INTEGER NOT NULL,
    PRIMARY KEY (sno, pno));

--1と3、2と4が一致する
INSERT INTO SupParts VALUES(1, 01);
INSERT INTO SupParts VALUES(1, 02);
INSERT INTO SupParts VALUES(1, 03);

INSERT INTO SupParts VALUES(2, 01);
INSERT INTO SupParts VALUES(2, 03);

INSERT INTO SupParts VALUES(3, 01);
INSERT INTO SupParts VALUES(3, 02);
INSERT INTO SupParts VALUES(3, 03);

INSERT INTO SupParts VALUES(4, 01);
INSERT INTO SupParts VALUES(4, 03);

INSERT INTO SupParts VALUES(5, 05);

INSERT INTO SupParts VALUES(6, 01);
INSERT INTO SupParts VALUES(6, 02);
select *
from Supparts;

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
13 rows affected.


sno,pno
1,1
1,2
1,3
2,1
2,3
3,1
3,2
3,3
4,1
4,3


# やりたいこと
+ 集合の相等チェック
    + 数も種類も全く同じ部品を取り扱う供給業者のペアを見つける

In [208]:
%%sql
select distinct s1.sno, s2.sno
from Supparts as s1, Supparts as s2
where s1.sno < s2.sno and
-- 「S1 の方がS2より真に大きい」でないか
not exists (
    select *
    from Supparts as s3
    where s1.sno = s3.sno and
    not exists (
        select *
        from Supparts as s4
        where s2.sno = s4.sno and
        s3.pno = s4.pno
    )
-- 「S2の方がS1より真に大きい」でないか
) and not exists (
    select *
    from Supparts as s3
    where s2.sno = s3.sno and
    not exists (
        select *
        from Supparts as s4
        where s1.sno = s4.sno and
        s3.pno = s4.pno
    )
)

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


sno,sno_1
1,3
2,4


In [213]:
%%sql
select distinct s1.sno, s2.sno
from Supparts as s1, Supparts as s2
where s1.sno < s2.sno
-- 「S1 の方がS2より真に大きい」でないか
and not exists (
    select s3.pno
    from Supparts as s3
    where s1.sno = s3.sno
    except
    select s3.pno
    from Supparts as s3
    where s2.sno = s3.sno
)
-- 「S2の方がS1より真に大きい」でないか
and not exists (
    select s3.pno
    from Supparts as s3
    where s2.sno = s3.sno
    except
    select s3.pno
    from Supparts as s3
    where s1.sno = s3.sno
)

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


sno,sno_1
1,3
2,4


In [193]:
%%sql
select s1.sno, s2.sno
from SupParts as s1, Supparts as s2, (
    select sno, count(*)
    from Supparts
    group by sno
) as s3, (
    select sno, count(*)
    from Supparts
    group by sno
) as s4
where s1.sno < s2.sno and s1.pno = s2.pno and
s1.sno = s3.sno and s2.sno = s4.sno
group by s1.sno, s2.sno
having count(*) = min(s3.count) and
count(*) = min(s4.count)

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


sno,sno_1
2,4
1,3


In [183]:
%%sql
-- 関係除算と似た考えで,havingをつかう
select s1.sno, s2.sno
from SupParts as s1, Supparts as s2
where s1.sno < s2.sno and s1.pno = s2.pno
group by s1.sno, s2.sno
having count(*) = (
    select count(*)
    from Supparts as s3
    where s2.sno = s3.sno
    group by sno
) and count(*) = (
    select count(*)
    from Supparts as s3
    where s1.sno = s3.sno
    group by sno
)

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


sno,sno_1
2,4
1,3


In [227]:
%%sql
select distinct s1.sno, s2.sno
from Supparts as s1, Supparts as s2
where s1.sno < s2.sno and
-- 空集合ならs3 = s4とわかる
-- 重なり(intersection) \subset 和集合の場合、一方が他方をはみ出している
not exists (
    (
        -- s3 \cup s4
        select s3.pno
        from Supparts as s3
        where s1.sno = s3.sno
        union
        select s4.pno
        from Supparts as s4
        where s2.sno = s4.sno
    ) except (
        -- s3 \cap s4(重なり)
        select s3.pno
        from Supparts as s3
        where s1.sno = s3.sno
        intersect
        select s4.pno
        from Supparts as s4
        where s2.sno = s4.sno
    )
)

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


sno,sno_1
1,3
2,4


In [210]:
%%sql
-- おまけ
select s1.sno, s2.sno
from Supparts as s1, Supparts as s2
where s1.sno < s2.sno and s1.pno = s2.pno and
(
    select count(*)
    from Supparts as s3
    where s3.sno = s1.sno
) = (
    select count(*)
    from Supparts as s3
    where s3.sno = s2.sno
)
group by s1.sno, s2.sno
having count(s1.sno || s2.sno) = (
    select count(*)
    from Supparts as s3
    where s3.sno = s1.sno
)

*  postgresql://padawan:***@db:5432/dsdojo_db
(psycopg2.errors.UndefinedFunction) 演算子が存在しません: integer || integer
LINE 14: having count(s1.sno || s2.sno) = (
                             ^
HINT:  指定した名称と引数の型に合う演算子がありません。明示的な型キャストが必要かもしれません。

[SQL: select s1.sno, s2.sno
from Supparts as s1, Supparts as s2
where s1.sno < s2.sno and s1.pno = s2.pno and
(
    select count(*)
    from Supparts as s3
    where s3.sno = s1.sno
) = (
    select count(*)
    from Supparts as s3
    where s3.sno = s2.sno
)
group by s1.sno, s2.sno
having count(s1.sno || s2.sno) = (
    select count(*)
    from Supparts as s3
    where s3.sno = s1.sno
)]
(Background on this error at: https://sqlalche.me/e/14/f405)
